# Filtering Echo Notes

### Based on the Echo Notes characterize whether a patient has a CHF or not
### In the Next Step Characterize whether Each patient has which level of CHF

In [88]:
## Importing LIbraries

from sqlalchemy import create_engine
import re
import pandas as pd
from pprint import pprint
import psycopg2
from collections import Counter
import random
from urllib.request import urlopen
import urllib.parse
import random
import json

In [54]:
%load_ext autoreload
%autoreload 2

### Connecting to Database to send the Echo Notes to Text Files

In [55]:
## Connecting to Database

conn_string = "host='localhost' dbname='mimic' user='mimicuser' password='1234567890'"
conn = psycopg2.connect(conn_string)

In [56]:
## Loading the Noteevents Database into df

sql_query = """select * from mimic.mimiciii.noteevents where category='Echo'"""
sql_query = re.sub(r'%', '%%', sql_query)
df = pd.read_sql_query(sql_query ,con=conn)
df.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,59653,31038,174978.0,2102-06-14,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Endocar...
1,59654,70150,156140.0,2191-05-10,None,None,Echo,Report,None,None,"PATIENT/TEST INFORMATION:\nIndication: CHF, A-..."
2,59655,54190,188571.0,2158-02-16,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Cerebro...
3,59656,5771,185291.0,2173-09-02,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: 78 year...
4,59657,80030,100442.0,2119-06-07,None,None,Echo,Report,None,None,PATIENT/TEST INFORMATION:\nIndication: Left ve...


## Writing Files into a Text Files in a Folder

In [26]:
for row_id,element in zip(df.row_id, df.text):
    row_id, element
    with open("./EchoNotes/" + str(row_id) + ".txt",'w') as file:
        file.write(element)

# Match the Queries with Elastic Search powered by Solr

### Elastic search is the state of the art in search engines GOOGLE, I used this actually match the queries given in the LV RV Dysfunction modified document with all the Echo Notes

In [22]:
## Creating a Map for the Echo Notes

EchoMap = {}
EchoMap["LVSF"] = {'normal':{},'mild_moderate':{},'severely':{}}
EchoMap["LVCF"] = {'normal':{},'mild_moderate':{},'severely':{}}
EchoMap["RVSF"] = {'normal':{},'mild_moderate':{},'severely':{}}
EchoMap["RVCF"] = {'normal':{},'mild_moderate':{},'severely':{}}

In [79]:
KeyToMap = {}
KeyToMap["LVSF"] = "Search terms used to categorize left ventricular systolic function"
KeyToMap["LVCF"] = "Search terms used to categorize left ventricular cavity function"
KeyToMap["RVSF"] = "Search terms used to categorize right ventricular systolic function"
KeyToMap["RVCF"] = "Search terms used to categorize right ventricular cavity function"

In [32]:
EchoMap["LVSF"]['normal'] = """Overall normal LVEF (>55%)
Normal LV wall thickness, cavity size and regional/global systolic function (LVEF >55%)
Low normal LVEF
Normal LV wall thickness, cavity size, and systolic function (LVEF>55%)
Mild symmetric LVH with normal cavity size and regional/global systolic function (LVEF>55%)
Overall left ventricular systolic function is normal (LVEF>55%)
Normal LV wall thickness, cavity size, and global systolic function (LVEF>55%)
Left ventricular wall thickness, cavity size, and systolic function are normal (LVEF>55%)
Mild symmetric LVH with normal cavity size and systolic function (LVEF>55%)
Mild symmetric LVH with normal cavity size and global systolic function (LVEF>55%)
Overall left ventricular systolic function is low normal (LVEF 50-55%)
There is mild symmetric left ventricular hypertrophy with normal cavity size and systolic function (LVEF>55%)
Normal LV wall thickness, cavity size and regional/global function (LVEF >55%)
Normal LV wall thickness, cavity size, and global function (LVEF>55%)
Left ventricular wall thickness, cavity size, and systolic function are normal
There is mild symmetric left ventricular hypertrophy with normal cavity size and systolic function
Low normal LVEF (50-55%)
Hyperdynamic LVEF >75%
Hyperdynamic LVEF
Left ventricular systolic function is hyperdynamic (EF>75%)
Hyperdynamic left ventricular systolic function
Hyperdynamic LVEF (>75%)"""

EchoMap["LVSF"]['mild_moderate'] = """Mildly depressed LVEF
Mild global LV hypokinesis
Depressed LVEF
Overall left ventricular systolic function is mildly depressed
depressed LVEF
Mildly depressed LVEF (40-55%)
global LV hypokinesis
LV systolic function appears depressed
There is mild global left ventricular hypokinesis
Intrinsic LV systolic function depressed given the severity of valvular regurgitation
Moderately depressed LVEF
Overall left ventricular systolic function is moderately depressed
Moderate global LV hypokinesis
There is moderate global left ventricular hypokinesis
Mild-moderate global left ventricular hypokinesis
Moderately depressed LVEF (30-39%)"""

EchoMap["LVSF"]['severely'] = """Severely depressed LVEF
Severe global LV hypokinesis
Overall left ventricular systolic function is severely depressed
There is severe global left ventricular hypokinesis
Moderate-severe global left ventricular hypokinesis
Severely depressed LVEF (<30%)
-severe global left ventricular hypokinesis"""

In [36]:
EchoMap["LVCF"]['normal'] = """Normal LV cavity size
Normal LV wall thickness, cavity size and regional/global systolic function (LVEF >55%)
The left ventricular cavity size is normal
Normal LV wall thickness and cavity size
Mild symmetric LVH with normal cavity size
Normal LV wall thickness, cavity size, and systolic function (LVEF>55%)
Mild symmetric LVH with normal cavity size and regional/global systolic function (LVEF>55%)
Normal LV wall thickness, cavity size, and global systolic function (LVEF>55%)
Left ventricular wall thickness, cavity size, and systolic function are normal (LVEF>55%)
Normal LV wall thicknesses and cavity size
Mild symmetric LVH with normal cavity size and systolic function (LVEF>55%)
Left ventricular wall thicknesses and cavity size are normal
Mild symmetric LVH with normal cavity size and global systolic function (LVEF>55%)
There is mild symmetric left ventricular hypertrophy with normal cavity size and systolic function (LVEF>55%)
There is mild symmetric left ventricular hypertrophy with normal cavity size
The left ventricular cavity size is top normal/borderline dilated
Normal LV wall thickness, cavity size, and global function (LVEF>55%)
Left ventricular wall thickness, cavity size, and systolic function are normal
LV cavity normal for BSA
There is mild symmetric left ventricular hypertrophy with normal cavity size and systolic function
Top /borderline dilated LV cavity size
Normal LV thickness and cavity size
Normal LV wall thickness, cavity size and regional/global function (LVEF >55%)"""

EchoMap["LVCF"]['mild_moderate'] = """Mildly dilated LV cavity
The left ventricular cavity is mildly dilated
Dilated LV cavity
The left ventricular cavity is dilated
Mildly dilated LV cavity
Moderately dilated LV cavity
The left ventricular cavity is moderately dilated"""

EchoMap["LVCF"]['severely'] = """Severely dilated LV cavity
The left ventricular cavity is severely dilated"""

In [35]:
EchoMap["RVSF"]['normal'] = """Normal RV chamber size and free wall motion
Right ventricular chamber size and free wall motion are normal
Normal RV systolic function
Right ventricular systolic function is borderline normal
Borderline normal RV systolic function
Right ventricular systolic function is normal
Right ventricular chamber size and systolic function are normal
TASPE normal (>=1.6cm) Normal interventricular septal motion
TASPE normal (>=1.6cm)
Normal RV chamber size and free motion
Normal RV function"""

EchoMap["RVSF"]['mild_moderate'] = """Mild global RV free wall hypokinesis
There is mild global right ventricular free wall hypokinesis
Right ventricular systolic function appears depressed
RV function depressed
Mild global RV free hypokinesis
TASPE depressed (<1.6cm)
TASPE depressed (<1.6cm) RV function depressed
TASPE depressed (<1.6cm) Abnormal systolic septal motion/position consistent with RV pressure overload
TASPE depressed (<1.6cm) Abnormal septal motion/position consistent with RV pressure/volume overload
TASPE depressed (<1.6cm) Intrinsic RV systolic function likely more depressed given the severity of TR
TASPE depressed (<1.6cm) Paradoxical septal motion consistent with conduction abnormality/ventricular pacing
TASPE depressed (<1.6cm) Normal interventricular septal motion
TASPE depressed (<1.6cm) Paradoxical septal motion consistent with prior cardiac surgery
TASPE depressed (<1.6cm) Abnormal septal motion/position
There is moderate global right ventricular free wall hypokinesis
Moderate global RV free wall hypokinesis
Moderate global RV free hypokinesis"""

EchoMap["RVSF"]['severely'] = """There is severe global right ventricular free wall hypokinesis
Severe global RV free wall hypokinesis"""

In [34]:
EchoMap["RVCF"]['normal'] = """Normal RV chamber size and free wall motion
Right ventricular chamber size and free wall motion are normal
Normal RV chamber size
Right ventricular chamber size is normal
Right ventricular chamber size and systolic function are normal
Normal RV chamber size and free motion
Right ventricular chamber size and free motion are normal"""

EchoMap["RVCF"]['mild_moderate'] = """Mildly dilated RV cavity
Dilated RV cavity
The right ventricular cavity is dilated
The right ventricular cavity is mildly dilated"""

EchoMap["RVCF"]['severely'] = """Moderately dilated RV cavity
Markedly dilated RV cavity
The right ventricular cavity is moderately dilated
The right ventricular cavity is markedly dilated"""

In [47]:
for key in (EchoMap):
    for level in EchoMap[key]:
        sents = EchoMap[key][level].split("\n")
        EchoMap[key][level] = {}
        for sent in sents:
            EchoMap[key][level][sent] = list()

In [ ]:
connection = urlopen('http://localhost:8983/solr/chf_sepsis/select?q="Left+ventricular+systolic+function+is+hyperdynamic+\(EF>75\)"~4&wt=python&rows=50000')
response = eval(connection.read())
print (response['response']['numFound'], "documents found.")
for i,document in enumerate(response['response']['docs']):
    if i == 10 or i == len(response['response']['docs']) - 1:
        print(i, document['id'].split('/')[-1])
        print()

In [69]:
url1 = 'http://localhost:8983/solr/chf_sepsis/select?q="'
url3 = '"~4&wt=python&rows=50000'
for key in (EchoMap):
    for level in EchoMap[key]:
        for sent in EchoMap[key][level]:
            url2 = sent.replace(' ','+').replace('(','\(').replace('%','').replace(')','\)')
            connection = urlopen(url1 + url2 + url3)
            response = eval(connection.read())
            for document in (response['response']['docs']):
                EchoMap[key][level][sent].append(int(document['id'].split('/')[-1][:-4]))

In [89]:
with open('EchoMap.json', 'w') as fp:
    json.dump(EchoMap, fp)

In [87]:
with open("StatisticOfSentences.txt",'w') as f:

    for key in (EchoMap):
        print(KeyToMap[key], file=f)
        print("",file=f)
        for level in EchoMap[key]:
            print('*'*25,file=f)
            print(level.upper(),file=f)
            print('*'*25,file=f)
            print("",file=f)
            for sent in EchoMap[key][level]:
                if EchoMap[key][level][sent]:
                    print(sent,file=f)
                    print("Number of Documents", len(EchoMap[key][level][sent]),file=f)
                    print("First", EchoMap[key][level][sent][0],file=f)
                    print("Random", random.choice(EchoMap[key][level][sent]),file=f)
                    print("Last", EchoMap[key][level][sent][-1],file=f)
                    print("",file=f)
                else:
                    print(sent,file=f)
                    print("Number of Documents", len(EchoMap[key][level][sent]),file=f)
                    print("First", 0,file=f)
                    print("Random", 0,file=f)
                    print("Last", 0,file=f)
                    print("",file=f)
        print("",file=f)
        print("*"*100,file=f)

In [101]:
CHF_Patients_severely = set()
CHF_Patients_mild_moderate = set()
CHF_Patients_severely_AND_mild_moderate = set()

for key in (EchoMap):
    for level in EchoMap[key]:
        if level in ['severely']:
            for sent in EchoMap[key][level]:
                CHF_Patients_severely |= set(EchoMap[key][level][sent])

for key in (EchoMap):
    for level in EchoMap[key]:
        if level in ['mild_moderate']:
            for sent in EchoMap[key][level]:
                CHF_Patients_mild_moderate |= set(EchoMap[key][level][sent])

for key in (EchoMap):
    for level in EchoMap[key]:
        if level in ['severely','mild_moderate']:
            for sent in EchoMap[key][level]:
                CHF_Patients_severely_AND_mild_moderate |= set(EchoMap[key][level][sent])

In [104]:
CHF_Patients_severely = df[df['row_id'].isin(CHF_Patients_severely)]

In [105]:
CHF_Patients_mild_moderate = df[df['row_id'].isin(CHF_Patients_mild_moderate)]

In [106]:
CHF_Patients_severely_AND_mild_moderate = df[df['row_id'].isin(CHF_Patients_severely_AND_mild_moderate)]

In [112]:
header = ["row_id", "subject_id", "hadm_id"]
CHF_Patients_severely.to_csv("CHF_Patients_severely.csv", columns=header)
CHF_Patients_mild_moderate.to_csv("CHF_Patients_mild_moderate.csv", columns=header)
CHF_Patients_severely_AND_mild_moderate.to_csv("CHF_Patients_severely_AND_mild_moderate.csv", columns=header)